In [101]:
import pandas as pd
import basedosdados as bd
import plotly.express as px

In [102]:
query = """
SELECT DISTINCT id_municipio, titulo_eleitoral, sigla_partido, data_filiacao, data_desfiliacao, motivo_cancelamento
FROM basedosdados.br_tse_filiacao_partidaria.microdados
WHERE id_municipio = '3304557'
"""

filiados = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 445923/445923 [00:36<00:00, 12244.59rows/s]


In [120]:
filiados['data_filiacao'] = filiados['data_filiacao'].astype(str)
filiados['data_desfiliacao'] = filiados['data_desfiliacao'].astype(str)

In [164]:
flt = ((filiados['data_filiacao'].str.contains('2020') |
      filiados['data_filiacao'].str.contains('2021')))

filiados_20_21 = filiados[flt].groupby('sigla_partido')['sigla_partido'].count().reset_index(name='count').sort_values(by='count', ascending=True)

filiados_20_21['%'] = filiados_20_21['count'] / filiados_20_21['count'].sum() * 100

fig = px.bar(filiados_20_21.tail(10).round(2),  
             x = '%', 
             y = 'sigla_partido',
             barmode = 'group',
             text = '%',
             color_discrete_sequence=['rgb(67,133,197)'],
             labels = {'sigla_partido': '', '%': ''})

fig.update_yaxes(showticklabels = True)
fig.update_xaxes(showticklabels = False, tickfont = {'size': 15}, range = [0, 40])
fig.update_layout(width = 500, height = 500, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}%', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/pct_filiados_20_21.png")

In [165]:
filiados[flt].value_counts('titulo_eleitoral')

titulo_eleitoral
075007760329    5
001712250302    4
053264340370    3
088349620345    3
086455660345    3
               ..
074324050353    1
074320150370    1
074301030396    1
074297260302    1
888497440302    1
Length: 7143, dtype: int64

In [166]:
flt = filiados['titulo_eleitoral'] == '001712250302'
filiados[flt].sort_values(by = 'data_filiacao', ascending = True)

,id_municipio,titulo_eleitoral,sigla_partido,data_filiacao,data_desfiliacao,motivo_cancelamento
200525,3304557,001712250302,PCDOB,1985-04-12,2013-05-20,JUDICIAL
134866,3304557,001712250302,PTB,2010-05-03,2012-05-16,JUDICIAL
253977,3304557,001712250302,PSDB,2012-03-15,2018-12-06,JUDICIAL
400290,3304557,001712250302,PTB,2012-06-07,None,CANCELAMENTO AUTOMÁTICO
261800,3304557,001712250302,PCDOB,2013-05-21,2018-10-24,CANCELAMENTO AUTOMÁTICO
300130,3304557,001712250302,PSB,2014-11-25,None,CANCELAMENTO AUTOMÁTICO
67550,3304557,001712250302,CIDADANIA,2016-04-02,None,CANCELAMENTO AUTOMÁTICO
28593,3304557,001712250302,PROS,2016-06-08,None,CANCELAMENTO AUTOMÁTICO
295246,3304557,001712250302,PSL,2018-01-27,None,CANCELAMENTO AUTOMÁTICO
232896,3304557,001712250302,PSOL,2020-01-21,None,CANCELAMENTO AUTOMÁTICO


In [154]:
filiados_20_21

,sigla_partido,count,%
24,PSTU,3,0.040984
28,PV,13,0.177596
7,PCB,26,0.355191
8,PCDOB,37,0.505464
29,REDE,56,0.765027
13,PMN,56,0.765027
9,PCO,59,0.806011
15,PP,74,1.010929
4,MDB,75,1.024590
18,PSB,76,1.038251


In [145]:
flt = ((filiados['data_filiacao'].str.contains('2020') == True) & 
      ((filiados['data_desfiliacao'].str.contains('2021') == True) | 
      (filiados['data_desfiliacao'].str.contains('2020') == True)))

# pessoas que se filiaram em 2020 e desfiliaram no mesmo ano ou em 2021
des_filiado_20_21 = filiados[flt]

por_partido = des_filiado_20_21.groupby('sigla_partido')['sigla_partido'].count().reset_index(name='count').sort_values(['count'], ascending=True)

por_partido['%'] = por_partido['count'] / por_partido['count'].sum() * 100

fig = px.bar(por_partido.tail(10).round(2),  
             x = '%', 
             y = 'sigla_partido',
             barmode = 'group',
             text = '%',
             color_discrete_sequence=['rgb(67,133,197)'],
             labels = {'sigla_partido': '', '%': ''})

fig.update_yaxes(showticklabels = True)
fig.update_xaxes(showticklabels = False, tickfont = {'size': 15}, range = [0, 120])
fig.update_layout(width = 500, height = 500, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}%', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/pct_filiado_20_desfiliado_20_21.png")

In [162]:
des_filiado_20_21

,id_municipio,titulo_eleitoral,sigla_partido,data_filiacao,data_desfiliacao,motivo_cancelamento,mes_desfiliacao
912,3304557,080804680337,PSL,2020-01-20,2021-04-08,None,2021-04
916,3304557,078051020396,PSL,2020-01-17,2021-04-09,None,2021-04
922,3304557,004646620329,PSL,2020-03-27,2021-04-09,None,2021-04
926,3304557,017286250353,PSL,2020-03-15,2021-04-05,None,2021-04
942,3304557,089463240329,PSL,2020-03-30,2020-09-25,A PEDIDO DO ELEITOR,2020-09
...,...,...,...,...,...,...,...
442156,3304557,078341320396,PSL,2020-01-17,2021-04-09,None,2021-04
442157,3304557,121211690302,PSL,2020-01-25,2021-04-07,None,2021-04
442164,3304557,097638720329,PSL,2020-03-16,2021-04-09,None,2021-04
442166,3304557,082126000337,PSL,2020-03-31,2021-04-08,None,2021-04


In [152]:
des_filiado_20_21['data_filiacao'] = pd.to_datetime(des_filiado_20_21['data_filiacao'], format='%Y-%m-%d')
des_filiado_20_21['data_desfiliacao'] = pd.to_datetime(des_filiado_20_21['data_desfiliacao'], format='%Y-%m-%d')

des_filiado_20_21['mes_desfiliacao'] = des_filiado_20_21['data_desfiliacao'].dt.to_period('M')
des_filiado_20_21['mes_desfiliacao'] = des_filiado_20_21['mes_desfiliacao'].astype(str)

flt = des_filiado_20_21['sigla_partido'] == 'PSL'

desfiliacao_PSL = des_filiado_20_21[flt].groupby('mes_desfiliacao')['mes_desfiliacao'].count().reset_index(name='count').sort_values(['mes_desfiliacao'], ascending=True)

fig = px.bar(desfiliacao_PSL.tail(10).round(2),  
             x = 'mes_desfiliacao', 
             y = 'count',
             barmode = 'group',
             text = 'count',
             color_discrete_sequence=['rgb(67,133,197)'],
             labels = {'count': '', 'mes_desfiliacao': ''})

fig.update_yaxes(showticklabels = False)
fig.update_xaxes(showticklabels = True, tickfont = {'size': 15})
fig.update_layout(width = 1000, height = 500, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/n_desfiliado_PSL_mes.png")

In [148]:
des_filiado_20_21

,id_municipio,titulo_eleitoral,sigla_partido,data_filiacao,data_desfiliacao,motivo_cancelamento,mes_desfiliacao
912,3304557,080804680337,PSL,2020-01-20,2021-04-08,None,2021-04
916,3304557,078051020396,PSL,2020-01-17,2021-04-09,None,2021-04
922,3304557,004646620329,PSL,2020-03-27,2021-04-09,None,2021-04
926,3304557,017286250353,PSL,2020-03-15,2021-04-05,None,2021-04
942,3304557,089463240329,PSL,2020-03-30,2020-09-25,A PEDIDO DO ELEITOR,2020-09
...,...,...,...,...,...,...,...
442156,3304557,078341320396,PSL,2020-01-17,2021-04-09,None,2021-04
442157,3304557,121211690302,PSL,2020-01-25,2021-04-07,None,2021-04
442164,3304557,097638720329,PSL,2020-03-16,2021-04-09,None,2021-04
442166,3304557,082126000337,PSL,2020-03-31,2021-04-08,None,2021-04
